# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [2]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.18
Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial
Kleio image: timelinkserver/kleio-server
Kleio server token: 91JBg...
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Kleio server container: dazzling_wilson
Kleio version requested: latest
Kleio server version: 12.7.579 (2025-01-29 17:45:15)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=p

TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [3]:
tlnb.table_row_count_df()

,table,count
0,acts,40
1,acusacoes,0
2,adendas,0
3,aforamentos,0
4,alembic_version,1
5,aregisters,0
6,attributes,15452
7,blinks,4
8,cartas,0
9,cartas_de_perdao,0


### Show the kleio files available 

In [6]:
kleio_files = tlnb.get_kleio_files()

file_colums = ['path','status','errors','warnings', 'translated',  'import_status', 'import_errors', 'import_warnings','imported']
kleio_files[file_colums]

,path,status,errors,warnings,translated,import_status,import_errors,import_warnings,imported
0,sources/auc-alunos.cli,V,0,0,2025-02-24 08:25:00+00:00,I,0,0,2025-02-24 08:25:58.862216
1,sources/b1685.cli,W,0,1,2025-02-24 08:23:00+00:00,I,0,0,2025-02-24 08:26:16.652570
2,sources/dehergne-a.cli,V,0,0,2025-02-24 08:24:00+00:00,E,3,0,2025-02-24 08:26:06.020325
3,sources/dehergne-locations-1644.cli,V,0,0,2025-02-24 08:23:00+00:00,I,0,0,2025-02-24 08:26:10.139477
4,sources/real-entities/real-entities.cli,V,0,0,2025-02-24 08:24:00+00:00,I,0,0,2025-02-24 08:26:12.939908


## Optionally clean translations 

In [5]:
tlnb.kleio_server.translation_clean("", recurse="yes")

['kleio/real-entities/real-entities.cli',
 'kleio/dehergne-locations-1644.cli',
 'kleio/dehergne-a.cli',
 'kleio/b1685.cli']


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [5]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()
# get the import status
import_status = tlnb.get_import_status()
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

INFO:root:Request translation of T sources/auc-alunos.cli
INFO:root:Request translation of T sources/b1685.cli
INFO:root:Request translation of T sources/dehergne-a.cli
INFO:root:Request translation of T sources/dehergne-locations-1644.cli
INFO:root:Request translation of T sources/real-entities/real-entities.cli
INFO:root:Importing sources/auc-alunos.cli
INFO:root:Importing sources/dehergne-a.cli
INFO:root:Storing 11 postponed relations
INFO:root:Importing sources/dehergne-locations-1644.cli
INFO:root:Importing sources/real-entities/real-entities.cli
INFO:root:Storing 18 postponed relations
INFO:root:Importing sources/b1685.cli
INFO:root:Storing 2 postponed relations


                     path            name    size directory  \
2  sources/dehergne-a.cli  dehergne-a.cli  119167   sources   

                          modified              modified_iso  \
2 2025-02-24 08:24:00.136480+00:00 2025-02-24 08:24:00+00:00   

       modified_string                     qtime         qtime_string  \
2  2025-02-24 08:24:00 1970-01-01 00:00:00+00:00  1970-01-01 00:00:00   

                             source_url  ...  \
2  /rest/sources/sources/dehergne-a.cli  ...   

                                             version  \
2  KleioTranslator - server version 12.7 - build ...   

                                rpt_url                               xml_url  \
2  /rest/reports/sources/dehergne-a.rpt  /rest/exports/sources/dehergne-a.xml   

   import_status  import_errors import_warnings  \
2              E              3               0   

                                    import_error_rpt import_warning_rpt  \
2  ERROR: dehergne-a.cli processing same_as re

## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

In [7]:
imported_files_df = tlnb.get_import_status()
imported_files_df[file_colums].sort_values("name")


KeyError: 'name'

### Check a translation report

If needed to check for errors.


In [8]:
file_number=1
rpt = tlnb.get_translation_report(imported_files_df, file_number)
print(rpt)


KleioTranslator - server version 12.6 - build 577 2024-10-24 16:53:53
6-12-2024 8-56

Processing data file dehergne-a.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 185
Obs: 
** Processing source fonte$dehergne-a
7: lista$dehergne-notices-a
*** End of File

Line 541 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-belchior-miguel-carneiro-leitao) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 814 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-jean-regis-lieou) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 876 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-alessandro-cicero) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.

Structure file: /usr/local/timelink/clio/src/gacto2.str
Structure processing report: /usr/local/timelink/clio/src/gacto2.srpt
Structure in JSON: /usr/lo

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [9]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

b1685.cli
No errors
No warnings


